In [ ]:
!pip install sentencepiece protobuf transformers scikit-learn --quiet

In [ ]:
import json
import logging
import random
import os
from datetime import datetime
from typing import List, Dict

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

print("All libraries imported successfully!")

All libraries imported successfully!


In [ ]:
# Aspect and Sentiment Mappings
ASPECT_MAP = {
    "Địa điểm": 1,
    "Kịch bản": 2,
    "Dàn dựng": 3,
    "Dàn cast": 4,
    "Khách mời": 5,
    "Khả năng chơi trò chơi": 6,
    "Quảng cáo": 7,
    "Thử thách": 8,
    "Tương tác giữa các thành viên": 9,
    "Tinh thần đồng đội": 10,
    "Khác": 0
}

SENTIMENT_MAP = {
    "tích cực": 1,
    "tiêu cực": 2,
    "trung tính": 0,
}

# Training Configuration
CONFIG = {
    # Data
    'data_file': './filtered.json',  # Sẽ được augment trong notebook này
    'max_len': 256,
    'max_quads': 4,

    # Model
    'model_name': 'vinai/phobert-base',
    'hidden_dim': 256,
    'num_aspects': 11,
    'num_sentiments': 3,

    # Training
    'batch_size': 8,
    'learning_rate': 2e-5,
    'weight_decay': 0.01,
    'epochs': 100,  # Giảm từ 200
    'early_stop_patience': 15,

    # Loss Configuration
    'label_smoothing': 0.1,
    'weight_entity': 2.0,
    'weight_opinion': 2.0,
    'weight_aspect': 1.5,
    'weight_sentiment': 1.0,

    # Augmentation
    'augmentation_factor': 1,  # Tạo 1 augmented version mỗi sample

    # Checkpoints
    'checkpoint_dir': './models/improved_checkpoints',
    'best_model_dir': './models/improved_best_model',

    # Device
    'device': 'cuda' if torch.cuda.is_available() else 'cpu'
}

print("Configuration:")
for key, value in CONFIG.items():
    print(f"  {key}: {value}")

Configuration:
  data_file: ./filtered.json
  max_len: 256
  max_quads: 4
  model_name: vinai/phobert-base
  hidden_dim: 256
  num_aspects: 11
  num_sentiments: 3
  batch_size: 8
  learning_rate: 2e-05
  weight_decay: 0.01
  epochs: 100
  early_stop_patience: 15
  label_smoothing: 0.1
  weight_entity: 2.0
  weight_opinion: 2.0
  weight_aspect: 1.5
  weight_sentiment: 1.0
  augmentation_factor: 1
  checkpoint_dir: ./models/improved_checkpoints
  best_model_dir: ./models/improved_best_model
  device: cuda


In [ ]:
# Vietnamese filler words và synonyms cho augmentation
FILLER_WORDS = [
    "thì", "mà", "nhỉ", "nhé", "nha", "ạ", "à", "ơi",
    "thật", "thực sự", "quá", "lắm", "vậy", "đấy"
]

SYNONYMS = {
    "hay": ["tốt", "đẹp", "ổn"],
    "tốt": ["hay", "đẹp", "ổn"],
    "đẹp": ["hay", "tốt", "ổn"],
    "xấu": ["tệ", "dở"],
    "tệ": ["xấu", "dở"],
    "thích": ["yêu", "mê"],
    "ghét": ["không thích", "chán"],
    "vui": ["vui vẻ", "tích cực"],
    "buồn": ["buồn tẻ", "chán"],
}

SENTENCE_STARTERS = [
    "Theo mình thì",
    "Cá nhân mình nghĩ",
    "Mình thấy",
    "Mình nghĩ",
]

SENTENCE_ENDERS = ["nhé", "nha", "đấy", "mà", "ạ"]


def synonym_replacement(text: str, labels: List[Dict], num_replacements: int = 2) -> str:
    """
    Thay thế từ bằng từ đồng nghĩa, tránh entity/opinion spans
    """
    words = text.split()
    new_words = words.copy()

    # Tìm các vị trí an toàn (không nằm trong entity/opinion)
    safe_indices = []
    for i, word in enumerate(words):
        is_safe = True
        for label in labels:
            entity = label.get('entity', '')
            opinion = label.get('opinion', '')

            if entity and entity.lower() != "implicit" and word.lower() in entity.lower():
                is_safe = False
                break
            if opinion and word.lower() in opinion.lower():
                is_safe = False
                break

        if is_safe:
            safe_indices.append(i)

    # Thay thế random
    if safe_indices:
        num_to_replace = min(num_replacements, len(safe_indices))
        indices_to_replace = random.sample(safe_indices, num_to_replace)

        for idx in indices_to_replace:
            word = words[idx].lower()
            if word in SYNONYMS:
                new_words[idx] = random.choice(SYNONYMS[word])

    return ' '.join(new_words)


def random_insertion(text: str, num_insertions: int = 1) -> str:
    """
    Chèn random filler words
    """
    words = text.split()

    for _ in range(num_insertions):
        if len(words) == 0:
            break
        insert_pos = random.randint(0, len(words))
        filler = random.choice(FILLER_WORDS)
        words.insert(insert_pos, filler)

    return ' '.join(words)


def context_expansion(text: str) -> str:
    """
    Thêm context ở đầu hoặc cuối câu
    """
    choice = random.choice(['start', 'end', 'both'])

    if choice == 'start':
        starter = random.choice(SENTENCE_STARTERS)
        return f"{starter} {text}"
    elif choice == 'end':
        ender = random.choice(SENTENCE_ENDERS)
        return f"{text} {ender}"
    else:
        starter = random.choice(SENTENCE_STARTERS)
        ender = random.choice(SENTENCE_ENDERS)
        return f"{starter} {text} {ender}"


def augment_sample(sample: Dict, augmentation_factor: int = 1) -> List[Dict]:
    """
    Augment một sample thành nhiều versions
    """
    augmented_samples = []
    text = sample['text']
    labels = sample['labels']

    for _ in range(augmentation_factor):
        aug_choice = random.choice(['synonym', 'insertion', 'context', 'none'])

        new_text = text

        if aug_choice == 'synonym':
            new_text = synonym_replacement(text, labels, num_replacements=random.randint(1, 2))
        elif aug_choice == 'insertion':
            new_text = random_insertion(text, num_insertions=random.randint(1, 2))
        elif aug_choice == 'context':
            new_text = context_expansion(text)

        # Verify entity/opinion vẫn tồn tại trong augmented text
        valid = True
        for label in labels:
            entity = label.get('entity', '')
            opinion = label.get('opinion', '')

            if entity and entity.lower() != "implicit" and entity.lower() not in new_text.lower():
                valid = False
                break
            if opinion and opinion.lower() not in new_text.lower():
                valid = False
                break

        if valid or aug_choice == 'none':
            augmented_samples.append({
                'text': new_text,
                'labels': labels
            })

    return augmented_samples


def augment_dataset(data: List[Dict], augmentation_factor: int = 1) -> List[Dict]:
    """
    Augment toàn bộ dataset
    """
    augmented_data = []

    # Giữ tất cả original samples
    augmented_data.extend(data)

    print(f"[AUGMENTATION] Processing {len(data)} samples...")

    for i, sample in enumerate(data):
        if i % 1000 == 0:
            print(f"  Progress: {i}/{len(data)}")

        aug_samples = augment_sample(sample, augmentation_factor)
        augmented_data.extend(aug_samples)

    print(f"[SUCCESS] Augmentation completed!")
    print(f"  Original: {len(data)}")
    print(f"  Total: {len(augmented_data)} (+{len(augmented_data) - len(data)})")

    return augmented_data

print("Data augmentation functions loaded!")

Data augmentation functions loaded!


In [ ]:
def find_span_indices(text, phrase):
    """
    Tìm vị trí bắt đầu và kết thúc của phrase trong text
    """
    if not phrase or phrase.lower() == "null" or phrase.lower() == "implicit" or phrase == "":
        return (-1, -1)

    start_idx = text.lower().find(phrase.lower())
    if start_idx == -1:
        return None

    end_idx = start_idx + len(phrase)
    return (start_idx, end_idx)


def process_json_data(file_path):
    """
    Load và process JSON data
    """
    print(f"[LOADING] Reading data from {file_path}...")

    with open(file_path, 'r', encoding='utf-8') as f:
        full_content = f.read()

    raw_json = json.loads(full_content)
    raw_entries = raw_json.get("results", [])

    processed_data = []

    for i, entry in enumerate(raw_entries):
        try:
            text = entry["text"]
            labels = entry["labels"]
        except KeyError as e:
            logger.warning(f"Sample {i+1}: Missing key {e}")
            continue

        valid_labels = []
        for label in labels:
            entity_span = find_span_indices(text, label['entity'])
            opinion_span = find_span_indices(text, label['opinion'])

            if entity_span is None:
                logger.warning(f"Sample {i+1}: Entity '{label.get('entity')}' not found")
                continue
            if opinion_span is None:
                logger.warning(f"Sample {i+1}: Opinion '{label.get('opinion')}' not found")
                continue

            valid_labels.append({
                "entity_text": label['entity'],
                "entity_span": entity_span,
                "opinion_text": label['opinion'],
                "opinion_span": opinion_span,
                "aspect": label['aspect'],
                "sentiment": label['sentiment']
            })

        if valid_labels:
            processed_data.append({
                "text": text,
                "labels": valid_labels
            })

    print(f"[SUCCESS] Loaded {len(processed_data)} samples")
    return processed_data

print("Data processing functions loaded!")

Data processing functions loaded!


In [ ]:
data = process_json_data(CONFIG['data_file'])

[LOADING] Reading data from ./filtered.json...


Streaming output truncated to the last 5000 lines.


[SUCCESS] Loaded 25068 samples


In [ ]:
# Split 80/20
train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

print(f"[SPLIT] Data split completed:")
print(f"  Training samples: {len(train_data)}")
print(f"  Validation samples: {len(val_data)}")

[SPLIT] Data split completed:
  Training samples: 20054
  Validation samples: 5014


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(CONFIG['model_name'], use_fast=True)
print("PhoBERT tokenizer loaded successfully!")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

bpe.codes: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

PhoBERT tokenizer loaded successfully!


## 🗂️ Dataset Class

In [ ]:
class EAOSDatasetManual(Dataset):
    def __init__(self, data, tokenizer, max_len=256, max_quads=4):
        self.data = data
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.max_quads = max_quads

    def __len__(self):
        return len(self.data)

    def manual_find_token_span(self, encoding, char_span):
        """
        Map character span to token span
        """
        char_start, char_end = char_span
        if char_start == -1:
            return 0, 0

        tokens = self.tokenizer.convert_ids_to_tokens(encoding['input_ids'][0])
        current_char_idx = 0
        token_spans = []

        for token in tokens:
            if token in [self.tokenizer.bos_token, self.tokenizer.eos_token, self.tokenizer.pad_token]:
                token_len = 0
            else:
                clean_token = token.replace('@@', '').replace('_', ' ')
                token_len = len(clean_token)

            start = current_char_idx
            end = current_char_idx + token_len
            token_spans.append((start, end))
            current_char_idx += token_len

        token_start, token_end = -1, -1

        for idx, (t_start, t_end) in enumerate(token_spans):
            if t_start <= char_start < t_end:
                token_start = idx
            if t_start < char_end <= t_end:
                token_end = idx

        if token_start != -1 and token_end == -1:
            token_end = token_start

        return (token_start, token_end) if (token_start != -1) else (0, 0)

    def __getitem__(self, idx):
        item = self.data[idx]
        text = item['text']
        labels = item['labels']

        encoding = self.tokenizer(
            text,
            padding='max_length',
            truncation=True,
            max_length=self.max_len,
            return_tensors='pt'
        )

        target_matrix = np.full((self.max_quads, 6), -1, dtype=int)

        for i, label in enumerate(labels):
            if i >= self.max_quads:
                break

            e_s, e_e = self.manual_find_token_span(encoding, label['entity_span'])
            o_s, o_e = self.manual_find_token_span(encoding, label['opinion_span'])

            asp_id = ASPECT_MAP.get(label['aspect'], 0)
            sent_id = SENTIMENT_MAP.get(label['sentiment'], 0)

            target_matrix[i] = [e_s, e_e, o_s, o_e, asp_id, sent_id]

        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'targets': torch.tensor(target_matrix, dtype=torch.long)
        }


# Create datasets
train_dataset = EAOSDatasetManual(train_data, tokenizer, CONFIG['max_len'], CONFIG['max_quads'])
val_dataset = EAOSDatasetManual(val_data, tokenizer, CONFIG['max_len'], CONFIG['max_quads'])

print(f"[DATASET] Created successfully:")
print(f"  Train dataset: {len(train_dataset)} samples")
print(f"  Val dataset: {len(val_dataset)} samples")

[DATASET] Created successfully:
  Train dataset: 20054 samples
  Val dataset: 5014 samples


In [ ]:
class MultiEAOSModel(nn.Module):
    def __init__(self, model_name="vinai/phobert-base",
                 num_aspects=11,
                 num_sentiments=3,
                 max_len=256,
                 max_quads=4,
                 hidden_dim=256):
        super(MultiEAOSModel, self).__init__()

        # PhoBERT Encoder
        self.bert = AutoModel.from_pretrained(model_name)
        self.bert_hidden_size = self.bert.config.hidden_size

        # Transformer Encoder
        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(
                d_model=self.bert_hidden_size,
                nhead=4,
                dim_feedforward=hidden_dim,
            ),
            num_layers=2
        )

        # Learnable Queries
        self.quad_queries = nn.Parameter(torch.randn(max_quads, self.bert_hidden_size))

        # Multi-Head Attention
        self.attention = nn.MultiheadAttention(
            embed_dim=self.bert_hidden_size,
            num_heads=4,
            batch_first=True
        )

        # Prediction Heads
        self.fc_e_start = nn.Linear(self.bert_hidden_size, max_len)
        self.fc_e_end = nn.Linear(self.bert_hidden_size, max_len)
        self.fc_o_start = nn.Linear(self.bert_hidden_size, max_len)
        self.fc_o_end = nn.Linear(self.bert_hidden_size, max_len)
        self.fc_aspect = nn.Linear(self.bert_hidden_size, num_aspects)
        self.fc_sentiment = nn.Linear(self.bert_hidden_size, num_sentiments)

    def forward(self, input_ids, attention_mask):
        # Encoding
        bert_out = self.bert(input_ids=input_ids, attention_mask=attention_mask)[0]

        # Transformer
        transformer_out = self.transformer(bert_out)

        # Multi-EAOS Decoding
        batch_size = input_ids.size(0)
        queries = self.quad_queries.unsqueeze(0).expand(batch_size, -1, -1)

        attn_out, _ = self.attention(query=queries, key=transformer_out, value=transformer_out)

        # Predictions
        e_start_logits = self.fc_e_start(attn_out)
        e_end_logits = self.fc_e_end(attn_out)
        o_start_logits = self.fc_o_start(attn_out)
        o_end_logits = self.fc_o_end(attn_out)
        aspect_logits = self.fc_aspect(attn_out)
        sentiment_logits = self.fc_sentiment(attn_out)

        return {
            "e_start": e_start_logits,
            "e_end": e_end_logits,
            "o_start": o_start_logits,
            "o_end": o_end_logits,
            "aspect": aspect_logits,
            "sentiment": sentiment_logits
        }

print("Model architecture defined!")

Model architecture defined!


In [ ]:
class LabelSmoothingCrossEntropy(nn.Module):
    def __init__(self, smoothing=0.1, ignore_index=-1):
        super().__init__()
        self.smoothing = smoothing
        self.ignore_index = ignore_index
        self.confidence = 1.0 - smoothing

    def forward(self, pred, target):
        pred = pred.log_softmax(dim=-1)

        valid_mask = target != self.ignore_index

        if valid_mask.sum() == 0:
            return torch.tensor(0.0, device=pred.device)

        n_classes = pred.size(-1)

        with torch.no_grad():
            # FIX: Thay -1 thành 0 trước khi scatter (tránh index âm)
            target = target.clone()
            target[~valid_mask] = 0  # Thay -1 → 0

            true_dist = torch.zeros_like(pred)
            true_dist.fill_(self.smoothing / (n_classes - 1))
            true_dist.scatter_(1, target.unsqueeze(1), self.confidence)

            # Zero out invalid positions
            true_dist[~valid_mask] = 0

        loss = torch.sum(-true_dist * pred, dim=-1)
        loss = loss[valid_mask].mean()

        return loss

class WeightedMultiEAOSLoss(nn.Module):
    """
    Weighted Loss với Label Smoothing
    """
    def __init__(self,
                 smoothing=0.1,
                 weight_entity=2.0,
                 weight_opinion=2.0,
                 weight_aspect=1.5,
                 weight_sentiment=1.0):
        super().__init__()
        self.criterion = LabelSmoothingCrossEntropy(smoothing=smoothing, ignore_index=-1)

        self.weight_entity = weight_entity
        self.weight_opinion = weight_opinion
        self.weight_aspect = weight_aspect
        self.weight_sentiment = weight_sentiment

    def forward(self, outputs, targets):
        # Flatten
        e_start_logits = outputs['e_start'].view(-1, outputs['e_start'].shape[-1])
        e_end_logits = outputs['e_end'].view(-1, outputs['e_end'].shape[-1])
        o_start_logits = outputs['o_start'].view(-1, outputs['o_start'].shape[-1])
        o_end_logits = outputs['o_end'].view(-1, outputs['o_end'].shape[-1])
        aspect_logits = outputs['aspect'].view(-1, outputs['aspect'].shape[-1])
        sentiment_logits = outputs['sentiment'].view(-1, outputs['sentiment'].shape[-1])

        e_start_target = targets[:, :, 0].reshape(-1)
        e_end_target = targets[:, :, 1].reshape(-1)
        o_start_target = targets[:, :, 2].reshape(-1)
        o_end_target = targets[:, :, 3].reshape(-1)
        aspect_target = targets[:, :, 4].reshape(-1)
        sentiment_target = targets[:, :, 5].reshape(-1)

        # Calculate weighted losses
        loss_e_start = self.criterion(e_start_logits, e_start_target) * self.weight_entity
        loss_e_end = self.criterion(e_end_logits, e_end_target) * self.weight_entity
        loss_o_start = self.criterion(o_start_logits, o_start_target) * self.weight_opinion
        loss_o_end = self.criterion(o_end_logits, o_end_target) * self.weight_opinion
        loss_aspect = self.criterion(aspect_logits, aspect_target) * self.weight_aspect
        loss_sentiment = self.criterion(sentiment_logits, sentiment_target) * self.weight_sentiment

        total_loss = (loss_e_start + loss_e_end +
                     loss_o_start + loss_o_end +
                     loss_aspect + loss_sentiment)

        component_losses = {
            'entity_loss': (loss_e_start + loss_e_end).item(),
            'opinion_loss': (loss_o_start + loss_o_end).item(),
            'aspect_loss': loss_aspect.item(),
            'sentiment_loss': loss_sentiment.item()
        }

        return total_loss, component_losses

print("Improved loss functions defined!")

Improved loss functions defined!


In [ ]:
def calculate_component_metrics(outputs, targets):
    """
    Tính F1, Precision, Recall cho từng component
    """
    batch_size, max_quads = targets.shape[0], targets.shape[1]

    # Get predictions
    pred_e_start = torch.argmax(outputs['e_start'], dim=-1).view(-1)
    pred_e_end = torch.argmax(outputs['e_end'], dim=-1).view(-1)
    pred_o_start = torch.argmax(outputs['o_start'], dim=-1).view(-1)
    pred_o_end = torch.argmax(outputs['o_end'], dim=-1).view(-1)
    pred_aspect = torch.argmax(outputs['aspect'], dim=-1).view(-1)
    pred_sentiment = torch.argmax(outputs['sentiment'], dim=-1).view(-1)

    # Get ground truth
    true_e_start = targets[:, :, 0].view(-1)
    true_e_end = targets[:, :, 1].view(-1)
    true_o_start = targets[:, :, 2].view(-1)
    true_o_end = targets[:, :, 3].view(-1)
    true_aspect = targets[:, :, 4].view(-1)
    true_sentiment = targets[:, :, 5].view(-1)

    # Valid mask
    valid_mask = (true_e_start != -1)

    if valid_mask.sum() == 0:
        return {
            'entity_span': {'f1': 0.0, 'precision': 0.0, 'recall': 0.0},
            'opinion_span': {'f1': 0.0, 'precision': 0.0, 'recall': 0.0},
            'aspect': {'f1': 0.0, 'precision': 0.0, 'recall': 0.0},
            'sentiment': {'f1': 0.0, 'precision': 0.0, 'recall': 0.0},
            'full_quad': {'f1': 0.0, 'precision': 0.0, 'recall': 0.0}
        }

    def calc_metrics(pred_correct):
        tp = (pred_correct & valid_mask).sum().item()
        fp = (~pred_correct & valid_mask).sum().item()
        fn = (~pred_correct & valid_mask).sum().item()

        precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
        f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0.0

        return {'f1': f1, 'precision': precision, 'recall': recall}

    # Calculate for each component
    entity_correct = (pred_e_start == true_e_start) & (pred_e_end == true_e_end)
    opinion_correct = (pred_o_start == true_o_start) & (pred_o_end == true_o_end)
    aspect_correct = (pred_aspect == true_aspect)
    sentiment_correct = (pred_sentiment == true_sentiment)

    full_quad_correct = (
        entity_correct & opinion_correct &
        aspect_correct & sentiment_correct
    )

    return {
        'entity_span': calc_metrics(entity_correct),
        'opinion_span': calc_metrics(opinion_correct),
        'aspect': calc_metrics(aspect_correct),
        'sentiment': calc_metrics(sentiment_correct),
        'full_quad': calc_metrics(full_quad_correct)
    }

print("Component-wise metrics function defined!")

Component-wise metrics function defined!


In [ ]:
def train_epoch(model, data_loader, optimizer, loss_fn, device):
    """
    Train một epoch với component-wise metrics
    """
    model.train()
    total_loss = 0

    all_metrics = {
        'entity_span': {'f1': 0, 'precision': 0, 'recall': 0},
        'opinion_span': {'f1': 0, 'precision': 0, 'recall': 0},
        'aspect': {'f1': 0, 'precision': 0, 'recall': 0},
        'sentiment': {'f1': 0, 'precision': 0, 'recall': 0},
        'full_quad': {'f1': 0, 'precision': 0, 'recall': 0}
    }

    all_component_losses = {
        'entity_loss': 0,
        'opinion_loss': 0,
        'aspect_loss': 0,
        'sentiment_loss': 0
    }

    progress_bar = tqdm(data_loader, desc="Training")

    for batch in progress_bar:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        targets = batch['targets'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask)

        loss, component_losses = loss_fn(outputs, targets)
        loss.backward()
        optimizer.step()

        metrics = calculate_component_metrics(outputs, targets)

        # Aggregate
        for component in all_metrics:
            for metric_name in all_metrics[component]:
                all_metrics[component][metric_name] += metrics[component][metric_name]

        for loss_name in all_component_losses:
            all_component_losses[loss_name] += component_losses[loss_name]

        total_loss += loss.item()

        progress_bar.set_postfix({
            'loss': f"{loss.item():.4f}",
            'quad_f1': f"{metrics['full_quad']['f1']:.4f}"
        })

    # Average
    num_batches = len(data_loader)
    avg_loss = total_loss / num_batches

    avg_metrics = {}
    for component in all_metrics:
        avg_metrics[component] = {
            metric: val / num_batches
            for metric, val in all_metrics[component].items()
        }

    avg_component_losses = {
        loss_name: val / num_batches
        for loss_name, val in all_component_losses.items()
    }

    return avg_loss, avg_metrics, avg_component_losses


def validate_epoch(model, data_loader, loss_fn, device):
    """
    Validate với component-wise metrics
    """
    model.eval()
    total_loss = 0

    all_metrics = {
        'entity_span': {'f1': 0, 'precision': 0, 'recall': 0},
        'opinion_span': {'f1': 0, 'precision': 0, 'recall': 0},
        'aspect': {'f1': 0, 'precision': 0, 'recall': 0},
        'sentiment': {'f1': 0, 'precision': 0, 'recall': 0},
        'full_quad': {'f1': 0, 'precision': 0, 'recall': 0}
    }

    all_component_losses = {
        'entity_loss': 0,
        'opinion_loss': 0,
        'aspect_loss': 0,
        'sentiment_loss': 0
    }

    progress_bar = tqdm(data_loader, desc="Validation")

    with torch.no_grad():
        for batch in progress_bar:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            targets = batch['targets'].to(device)

            outputs = model(input_ids, attention_mask)
            loss, component_losses = loss_fn(outputs, targets)

            metrics = calculate_component_metrics(outputs, targets)

            for component in all_metrics:
                for metric_name in all_metrics[component]:
                    all_metrics[component][metric_name] += metrics[component][metric_name]

            for loss_name in all_component_losses:
                all_component_losses[loss_name] += component_losses[loss_name]

            total_loss += loss.item()

            progress_bar.set_postfix({
                'val_loss': f"{loss.item():.4f}",
                'quad_f1': f"{metrics['full_quad']['f1']:.4f}"
            })

    # Average
    num_batches = len(data_loader)
    avg_loss = total_loss / num_batches

    avg_metrics = {}
    for component in all_metrics:
        avg_metrics[component] = {
            metric: val / num_batches
            for metric, val in all_metrics[component].items()
        }

    avg_component_losses = {
        loss_name: val / num_batches
        for loss_name, val in all_component_losses.items()
    }

    return avg_loss, avg_metrics, avg_component_losses


def print_metrics(epoch, train_loss, train_metrics, val_loss, val_metrics,
                 train_comp_losses=None, val_comp_losses=None):
    """
    Print metrics một cách đẹp mắt
    """
    print(f"\n{'='*80}")
    print(f"Epoch {epoch} Results:")
    print(f"{'='*80}")

    print(f"\nOverall Loss:")
    print(f"  Train: {train_loss:.4f}  |  Val: {val_loss:.4f}")

    if train_comp_losses and val_comp_losses:
        print(f"\nComponent Losses:")
        print(f"  Entity:    Train={train_comp_losses['entity_loss']:.4f}  |  Val={val_comp_losses['entity_loss']:.4f}")
        print(f"  Opinion:   Train={train_comp_losses['opinion_loss']:.4f}  |  Val={val_comp_losses['opinion_loss']:.4f}")
        print(f"  Aspect:    Train={train_comp_losses['aspect_loss']:.4f}  |  Val={val_comp_losses['aspect_loss']:.4f}")
        print(f"  Sentiment: Train={train_comp_losses['sentiment_loss']:.4f}  |  Val={val_comp_losses['sentiment_loss']:.4f}")

    print(f"\nF1 Scores (Component-wise):")
    components = ['entity_span', 'opinion_span', 'aspect', 'sentiment', 'full_quad']
    component_names = ['Entity Span', 'Opinion Span', 'Aspect', 'Sentiment', 'Full Quadruple']

    for comp, name in zip(components, component_names):
        train_f1 = train_metrics[comp]['f1']
        val_f1 = val_metrics[comp]['f1']
        print(f"  {name:15s}: Train={train_f1:.4f}  |  Val={val_f1:.4f}")

    print(f"{'='*80}\n")

print("Training/validation functions defined!")

Training/validation functions defined!


In [ ]:
def save_checkpoint(model, optimizer, epoch, train_loss, val_loss, val_metrics, filename):
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'train_loss': train_loss,
        'val_loss': val_loss,
        'val_metrics': val_metrics,
        'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    }
    torch.save(checkpoint, filename)
    print(f"[SAVED] Checkpoint: {filename}")


def save_best_model(model, epoch, val_loss, val_metrics, model_dir):
    os.makedirs(model_dir, exist_ok=True)

    model_path = os.path.join(model_dir, "model.pth")
    torch.save(model.state_dict(), model_path)

    config = {
        **{k: str(v) if k == 'device' else v for k, v in CONFIG.items()},
        'best_epoch': epoch,
        'best_val_loss': float(val_loss),
        'val_metrics': {
            comp: {k: float(v) for k, v in metrics.items()}
            for comp, metrics in val_metrics.items()
        },
        'aspect_map': ASPECT_MAP,
        'sentiment_map': SENTIMENT_MAP,
        'saved_at': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    }

    config_path = os.path.join(model_dir, "config.json")
    with open(config_path, 'w', encoding='utf-8') as f:
        json.dump(config, f, ensure_ascii=False, indent=2)

    print(f"[BEST MODEL] Saved at epoch {epoch}, Val Loss: {val_loss:.4f}, Full Quad F1: {val_metrics['full_quad']['f1']:.4f}")

# Create directories
os.makedirs(CONFIG['checkpoint_dir'], exist_ok=True)
os.makedirs(CONFIG['best_model_dir'], exist_ok=True)

print("Checkpoint management functions defined!")

Checkpoint management functions defined!


## 🚀 Initialize Model, Loss, Optimizer

In [ ]:
# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=CONFIG['batch_size'], shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=CONFIG['batch_size'], shuffle=False)

# Initialize model
model = MultiEAOSModel(
    model_name=CONFIG['model_name'],
    num_aspects=CONFIG['num_aspects'],
    num_sentiments=CONFIG['num_sentiments'],
    max_len=CONFIG['max_len'],
    max_quads=CONFIG['max_quads'],
    hidden_dim=CONFIG['hidden_dim']
).to(CONFIG['device'])

# Initialize improved loss with label smoothing and weighting
loss_fn = WeightedMultiEAOSLoss(
    smoothing=CONFIG['label_smoothing'],
    weight_entity=CONFIG['weight_entity'],
    weight_opinion=CONFIG['weight_opinion'],
    weight_aspect=CONFIG['weight_aspect'],
    weight_sentiment=CONFIG['weight_sentiment']
).to(CONFIG['device'])

# Optimizer
optimizer = optim.AdamW(
    model.parameters(),
    lr=CONFIG['learning_rate'],
    weight_decay=CONFIG['weight_decay']
)

print(f"\n[READY] Model, loss, and optimizer initialized!")
print(f"Device: {CONFIG['device']}")
print(f"Total parameters: {sum(p.numel() for p in model.parameters())}")

pytorch_model.bin:   0%|          | 0.00/543M [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/torch/nn/modules/transformer.py:392: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


model.safetensors:   0%|          | 0.00/543M [00:00<?, ?B/s]


[READY] Model, loss, and optimizer initialized!
Device: cuda
Total parameters: 143681294


## 🏃 MAIN TRAINING LOOP

In [ ]:
print("="*80)
print("TRAINING START")
print("="*80)

best_val_f1 = 0.0
patience_counter = 0

for epoch in range(1, CONFIG['epochs'] + 1):
    # Train
    train_loss, train_metrics, train_comp_losses = train_epoch(
        model, train_loader, optimizer, loss_fn, CONFIG['device']
    )

    # Validate
    val_loss, val_metrics, val_comp_losses = validate_epoch(
        model, val_loader, loss_fn, CONFIG['device']
    )

    # Print results
    print_metrics(
        epoch, train_loss, train_metrics, val_loss, val_metrics,
        train_comp_losses, val_comp_losses
    )

    # Save best model based on Full Quad F1
    current_val_f1 = val_metrics['full_quad']['f1']
    if current_val_f1 > best_val_f1:
        best_val_f1 = current_val_f1
        patience_counter = 0
        save_best_model(model, epoch, val_loss, val_metrics, CONFIG['best_model_dir'])
    else:
        patience_counter += 1
        print(f"[INFO] No improvement for {patience_counter}/{CONFIG['early_stop_patience']} epochs")

    # Early stopping
    if patience_counter >= CONFIG['early_stop_patience']:
        print(f"\n[EARLY STOPPING] Triggered at epoch {epoch}")
        print(f"Best Full Quad F1: {best_val_f1:.4f}")
        break

    # Save latest checkpoint
    latest_path = os.path.join(CONFIG['checkpoint_dir'], "latest_checkpoint.pth")
    save_checkpoint(model, optimizer, epoch, train_loss, val_loss, val_metrics, latest_path)

print("\n" + "="*80)
print("[TRAINING COMPLETED]")
print(f"Best Full Quad F1: {best_val_f1:.4f}")
print(f"Model saved to: {CONFIG['best_model_dir']}")
print("="*80)

TRAINING START


Validation: 100%|██████████| 627/627 [01:26<00:00,  7.25it/s, val_loss=27.5026, quad_f1=0.0000]



Epoch 1 Results:

Overall Loss:
  Train: 26.8545  |  Val: 24.1323

Component Losses:
  Entity:    Train=10.9530  |  Val=9.6061
  Opinion:   Train=12.1272  |  Val=11.0820
  Aspect:    Train=2.9150  |  Val=2.6876
  Sentiment: Train=0.8594  |  Val=0.7566

F1 Scores (Component-wise):
  Entity Span    : Train=0.3817  |  Val=0.4582
  Opinion Span   : Train=0.3027  |  Val=0.3200
  Aspect         : Train=0.3395  |  Val=0.4198
  Sentiment      : Train=0.6682  |  Val=0.7685
  Full Quadruple : Train=0.0280  |  Val=0.0464

[BEST MODEL] Saved at epoch 1, Val Loss: 24.1323, Full Quad F1: 0.0464
[SAVED] Checkpoint: ./models/improved_checkpoints/latest_checkpoint.pth


Validation: 100%|██████████| 627/627 [01:25<00:00,  7.32it/s, val_loss=25.3145, quad_f1=0.0000]



Epoch 2 Results:

Overall Loss:
  Train: 23.2118  |  Val: 22.5942

Component Losses:
  Entity:    Train=9.2762  |  Val=8.9256
  Opinion:   Train=10.6194  |  Val=10.4323
  Aspect:    Train=2.5781  |  Val=2.5068
  Sentiment: Train=0.7381  |  Val=0.7294

F1 Scores (Component-wise):
  Entity Span    : Train=0.4519  |  Val=0.4782
  Opinion Span   : Train=0.3383  |  Val=0.3425
  Aspect         : Train=0.4487  |  Val=0.4797
  Sentiment      : Train=0.7739  |  Val=0.7847
  Full Quadruple : Train=0.0557  |  Val=0.0666

[BEST MODEL] Saved at epoch 2, Val Loss: 22.5942, Full Quad F1: 0.0666
[SAVED] Checkpoint: ./models/improved_checkpoints/latest_checkpoint.pth


...


Epoch 37 Results:

Overall Loss:
  Train: 8.9771  |  Val: 21.3348

Component Losses:
  Entity:    Train=3.7801  |  Val=8.3851
  Opinion:   Train=3.8185  |  Val=10.1720
  Aspect:    Train=0.9472  |  Val=2.1473
  Sentiment: Train=0.4313  |  Val=0.6304

F1 Scores (Component-wise):
  Entity Span    : Train=0.9920  |  Val=0.6955
  Opinion Span   : Train=0.9908  |  Val=0.5949
  Aspect         : Train=0.9954  |  Val=0.7180
  Sentiment      : Train=0.9944  |  Val=0.8825
  Full Quadruple : Train=0.9757  |  Val=0.4302

[BEST MODEL] Saved at epoch 37, Val Loss: 21.3348, Full Quad F1: 0.4302
[SAVED] Checkpoint: ./models/improved_checkpoints/latest_checkpoint.pth


Validation: 100%|██████████| 627/627 [01:25<00:00,  7.29it/s, val_loss=23.4729, quad_f1=0.5000]



Epoch 38 Results:

Overall Loss:
  Train: 8.9551  |  Val: 21.1993

Component Losses:
  Entity:    Train=3.7652  |  Val=8.2690
  Opinion:   Train=3.8154  |  Val=10.1655
  Aspect:    Train=0.9436  |  Val=2.1312
  Sentiment: Train=0.4309  |  Val=0.6335

F1 Scores (Component-wise):
  Entity Span    : Train=0.9935  |  Val=0.7013
  Opinion Span   : Train=0.9909  |  Val=0.5900
  Aspect         : Train=0.9958  |  Val=0.7221
  Sentiment      : Train=0.9948  |  Val=0.8811
  Full Quadruple : Train=0.9781  |  Val=0.4315

[BEST MODEL] Saved at epoch 38, Val Loss: 21.1993, Full Quad F1: 0.4315
[SAVED] Checkpoint: ./models/improved_checkpoints/latest_checkpoint.pth


Training:  46%|████▋     | 1160/2507 [08:21<09:41,  2.31it/s, loss=8.8290, quad_f1=1.0000]


KeyboardInterrupt: 

## 🔍 Inference & Testing

In [ ]:
# Reverse mappings
ID2ASPECT = {v: k for k, v in ASPECT_MAP.items()}
ID2SENTIMENT = {v: k for k, v in SENTIMENT_MAP.items()}

def predict_text(model, tokenizer, text, device, max_len=256):
    """
    Dự đoán cho một câu văn bản
    """
    model.eval()

    inputs = tokenizer(
        text,
        padding='max_length',
        truncation=True,
        max_length=max_len,
        return_tensors="pt"
    )
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask)

    results = []

    pred_e_start = torch.argmax(outputs['e_start'], dim=-1)[0]
    pred_e_end = torch.argmax(outputs['e_end'], dim=-1)[0]
    pred_o_start = torch.argmax(outputs['o_start'], dim=-1)[0]
    pred_o_end = torch.argmax(outputs['o_end'], dim=-1)[0]
    pred_aspect = torch.argmax(outputs['aspect'], dim=-1)[0]
    pred_sent = torch.argmax(outputs['sentiment'], dim=-1)[0]

    tokens = tokenizer.convert_ids_to_tokens(input_ids[0])

    for i in range(len(pred_e_start)):
        e_s, e_e = pred_e_start[i].item(), pred_e_end[i].item()
        o_s, o_e = pred_o_start[i].item(), pred_o_end[i].item()

        # Filter invalid
        if e_s > e_e or o_s > o_e:
            continue
        if e_s == 0 or e_e == 0:
            continue
        if e_s >= len(tokens) or o_s >= len(tokens):
            continue

        entity_text = tokenizer.convert_tokens_to_string(
            tokens[e_s:e_e+1]
        ).replace('_', ' ').strip()

        opinion_text = tokenizer.convert_tokens_to_string(
            tokens[o_s:o_e+1]
        ).replace('_', ' ').strip()

        aspect_label = ID2ASPECT.get(pred_aspect[i].item(), "Khác")
        sentiment_label = ID2SENTIMENT.get(pred_sent[i].item(), "Trung tính")

        if entity_text and opinion_text:
            results.append({
                "entity": entity_text,
                "aspect": aspect_label,
                "opinion": opinion_text,
                "sentiment": sentiment_label
            })

    return results

print("Inference function loaded!")

Inference function loaded!


## 🧪 Test Inference

In [ ]:
# Test với một số câu mẫu
test_samples = [
    "Tôi thấy mùa 2 không hay bằng mùa 1 vì mùa 1 có Trấn Thành mùa 2 lại không có",
    "Chương trình rất hay, MC dẫn tốt",
    "Kịch bản nhạt nhẽo, dàn cast không ăn ý"
]

print("[INFERENCE TEST]\n")

for i, sample_text in enumerate(test_samples, 1):
    print(f"Sample {i}: {sample_text}")
    predictions = predict_text(model, tokenizer, sample_text, CONFIG['device'])

    if predictions:
        for j, pred in enumerate(predictions, 1):
            print(f"  Prediction {j}:")
            print(f"    Entity: {pred['entity']}")
            print(f"    Aspect: {pred['aspect']}")
            print(f"    Opinion: {pred['opinion']}")
            print(f"    Sentiment: {pred['sentiment']}")
    else:
        print("  No predictions")
    print()

[INFERENCE TEST]

Sample 1: Tôi thấy mùa 2 không hay bằng mùa 1 vì mùa 1 có Trấn Thành mùa 2 lại không có
  Prediction 1:
    Entity: mùa 2 không
    Aspect: Kịch bản
    Opinion: không hay bằng mùa
    Sentiment: tiêu cực
  Prediction 2:
    Entity: 2 lại không
    Aspect: Dàn cast
    Opinion: <s>
    Sentiment: tiêu cực
  Prediction 3:
    Entity: 2 lại không
    Aspect: Dàn cast
    Opinion: <s>
    Sentiment: tiêu cực
  Prediction 4:
    Entity: </s> <pad>
    Aspect: Dàn cast
    Opinion: <s>
    Sentiment: tiêu cực

Sample 2: Chương trình rất hay, MC dẫn tốt
  Prediction 1:
    Entity: Chương trình rất
    Aspect: Kịch bản
    Opinion: rất hay,
    Sentiment: tích cực
  Prediction 2:
    Entity: dẫn
    Aspect: Dàn cast
    Opinion: tốt
    Sentiment: tích cực
  Prediction 3:
    Entity: Chương trình rất
    Aspect: Kịch bản
    Opinion: <s>
    Sentiment: tích cực
  Prediction 4:
    Entity: Chương trình rất
    Aspect: Kịch bản
    Opinion: ,
    Sentiment: tích cực

Sample 3:

## 📊 Summary

### Cải tiến đã áp dụng:
1. ✅ **Data Augmentation**: Tăng dataset size
2. ✅ **Label Smoothing**: Giảm overfitting
3. ✅ **Weighted Loss**: Focus vào components quan trọng
4. ✅ **Component-wise F1**: Metrics chi tiết
5. ✅ **Reduced Epochs**: 100 epochs thay vì 200

### Model đã được lưu tại:
- Best model: `./models/improved_best_model/`
- Checkpoints: `./models/improved_checkpoints/`

### Để sử dụng model cho backend:
Load model từ `./models/improved_best_model/model.pth` và config từ `config.json`


In [ ]:
from google.colab import files
files.download('/content/models/improved_checkpoints/latest_checkpoint.pth')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>